**Initialization and Constant Definition**

This section initializes the basic settings of the game, including the window size (WIDTH, HEIGHT), the frame rate (FPS), the size of the bird and the pipes, and physics parameters such as gravity and the strength of the bird's jump. pygame.init() is used to initialize Pygame.

In [ ]:
import pygame
import random
import pickle
import os

# Initialize Pygame
pygame.init()

# Define constants
WHITE = (255, 255, 255)  # RGB color for white
WIDTH, HEIGHT = 400, 600  # Screen dimensions
screen = pygame.display.set_mode((WIDTH, HEIGHT))  # Create a Pygame screen
pygame.display.set_caption("Clumsy Bird Flies Early")  # Set window title

FPS = 30  # Frames per second
BIRD_SIZE = 20  # Bird dimensions
PIPE_WIDTH = 50  # Width of pipes
PIPE_GAP = 150  # Gap between the top and bottom pipes
GRAVITY = 1  # Gravity for bird's motion
FLAP_STRENGTH = -10  # Upward velocity when bird flaps

**Defines the Core Parameters of the Reinforcement Learning Algorithm**

ACTIONS: two actions of the bird (no jump or jump).

ALPHA: Learning rate, used to control the step size of the Q-value update.

GAMMA: Discount factor, which measures the weight of future rewards.

EPSILON: Exploration probability, controls the balance of exploration and utilization of the algorithm. EPSILON decreases (via EPSILON_DECAY) as training progresses, down to EPSILON_MIN.

In [ ]:
# Reinforcement learning parameters
ACTIONS = [0, 1]  # 0: Do nothing, 1: Flap
ALPHA = 0.2  # Learning rate
GAMMA = 0.9  # Discount factor for future rewards
EPSILON = 0.2  # Initial exploration probability
EPSILON_DECAY = 0.99  # Decay rate for exploration
EPSILON_MIN = 0.01  # Minimum exploration probability

**Q Table File Loading and Initialization**

The Q table is loaded from a file or initialized to an empty dictionary. If the file exists, the previously stored Q values are loaded; otherwise, an empty table is created. This part uses the pickle serialization technique.

In [ ]:
# Load or initialize Q-table
Q_FILE = "q_table.pkl"  # File to save Q-table
if os.path.exists(Q_FILE):  # Check if Q-table file exists
    with open(Q_FILE, "rb") as f:
        Q = pickle.load(f)  # Load Q-table from file
else:
    Q = {}  # Initialize empty Q-table

**Defining Auxiliary Functions**

These functions are used for reinforcement learning Q-table related operations:

get_state(): converts a bird's state into discrete state space.

get_q(): gets the state-action value from the Q-table, returning the default value of 0 if it does not exist.


set_q(): update the state-action values in the Q-table.

save_q_table(): save the Q table to a file to continue training for the next game.

In [ ]:
def get_state(bird_y, pipe_x, pipe_y, bird_velocity):
    return (bird_y // 10, pipe_x // 10, pipe_y // 10, int(bird_velocity))

def get_q(state, action):
    return Q.get((state, action), 0)  # Return 0 if the state-action pair is not in Q-table

def set_q(state, action, value):
    Q[(state, action)] = value

def save_q_table():
    with open(Q_FILE, "wb") as f:
        pickle.dump(Q, f)

**Load and Adjust Game Images**

Loads the image resources needed for the game and scales them to fit the screen size and the size of the game elements.

In [ ]:
background_img = pygame.image.load("background.png")
bird_img = pygame.image.load("bird.png")
pipe_top_img = pygame.image.load("pipe_top.png")
pipe_bottom_img = pygame.image.load("pipe_bottom.png")

background_img = pygame.transform.scale(background_img, (WIDTH, HEIGHT))
bird_img = pygame.transform.scale(bird_img, (BIRD_SIZE, BIRD_SIZE))
pipe_top_img = pygame.transform.scale(pipe_top_img, (PIPE_WIDTH, HEIGHT))
pipe_bottom_img = pygame.transform.scale(pipe_bottom_img, (PIPE_WIDTH, HEIGHT))

**Define the Bird**

The Bird class represents the bird in the game and contains the following functions:

Initialize position and speed.

Performs a jump (changes speed).

Update position (affected by gravity).

Draws an image of the bird.

In [ ]:
class Bird:
    def __init__(self):
        self.x = 50  # Initial x position of the bird
        self.y = HEIGHT // 2  # Initial y position of the bird
        self.velocity = 0  # Initial velocity of the bird

    def flap(self):
        self.velocity = FLAP_STRENGTH  # Apply upward velocity

    def update(self):
        self.velocity += GRAVITY  # Apply gravity
        self.y += self.velocity  # Update bird's position based on velocity

    def draw(self):
        screen.blit(bird_img, (self.x, self.y))  # Draw the bird on the screen

Define the Pipe
**加粗文字**
The Pipe class represents the pipes in the game and contains the following functions:

Initialize position (randomly generates intervals between upper and lower pipes).

Move the pipe (to the left).

Detect if the bird collides with the pipe (using rectangular collision detection).

In [ ]:
class Pipe:
    def __init__(self, x):
        self.x = x  # Initial x position of the pipe
        self.y = random.randint(100, HEIGHT - PIPE_GAP - 100)  # Random y position within valid range

    def update(self):
        self.x -= 5  # Move the pipe to the left

    def draw(self):
        # Draw the top and bottom pipes
        screen.blit(pipe_top_img, (self.x, self.y - pipe_top_img.get_height()))
        screen.blit(pipe_bottom_img, (self.x, self.y + PIPE_GAP))

    def collides_with(self, bird):
        # Check if the bird collides with the pipes
        bird_rect = pygame.Rect(bird.x, bird.y, BIRD_SIZE, BIRD_SIZE)  # Bird's rectangle
        top_pipe_rect = pygame.Rect(self.x, self.y - pipe_top_img.get_height(), PIPE_WIDTH, pipe_top_img.get_height())
        bottom_pipe_rect = pygame.Rect(self.x, self.y + PIPE_GAP, PIPE_WIDTH, HEIGHT - (self.y + PIPE_GAP))
        return bird_rect.colliderect(top_pipe_rect) or bird_rect.colliderect(bottom_pipe_rect)

**Define the Background**

The Background class handles the scrolling effect of the background to simulate a continuous motion scene.

In [ ]:
class Background:
    def __init__(self):
        self.x1 = 0  # x position of the first background
        self.x2 = WIDTH  # x position of the second background
        self.speed = 2  # Scrolling speed

    def update(self):
        self.x1 -= self.speed  # Move the first background to the left
        self.x2 -= self.speed  # Move the second background to the left
        # Reset background position when it goes offscreen
        if self.x1 + WIDTH < 0:
            self.x1 = self.x2 + WIDTH
        if self.x2 + WIDTH < 0:
            self.x2 = self.x1 + WIDTH

    def draw(self):
        # Draw the scrolling backgrounds
        screen.blit(background_img, (self.x1, 0))
        screen.blit(background_img, (self.x2, 0))

**Main Game Loop**

The main loop controls the game logic, including event handling, reinforcement learning Q-value updates, object position updates, collision detection, and drawing screen objects. The game continues to run until the user closes the window.

In [ ]:
def main():
    global EPSILON  # Use the global exploration probability
    bird = Bird()  # Create a bird object
    pipes = [Pipe(WIDTH + i * 200) for i in range(3)]  # Create initial pipes
    background = Background()  # Create a background object
    clock = pygame.time.Clock()  # Create a clock for controlling FPS
    score = 0  # Initialize score
    running = True  # Game running flag

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:  # Check for quit event
                running = False